# Victorian Liquor Licences by location

Data: https://www.data.vic.gov.au/data/dataset/victorian-liquor-licences-by-location

In [ ]:
#alcohol environment data for liveability indicators

In [1]:
import pandas as pd

In [3]:
locations = pd.read_excel("current_victorian_licences_by_location_january_2019.xlsx", skiprows = 3)

locations.head()

,Licence Num,Licensee,Trading As,Category,Trading Hours,After 11 pm,Maximum Capacity,Address,Suburb,Postcode,...,Longitude,Unnamed: 12,Unnamed: 13,Postal Address,Postal Suburb,Postal Postcode,Council,Region,Metro/Regional,Gaming
0,31100099,ARNOLD BRUCE,UNIVERSITY SKI CLUB,BYO Permit,BYO Permit,99,NaN,U S C LODGE ALLOTMENTS 15 & 16 ALPINE VILL,MT BULLER,3723.0,...,146.452276,NaN,NaN,PO BOX 93,SANDOWN VILLAGE,3171,MANSFIELD SHIRE COUNCIL,Hume,Regional,N
1,31100120,OAKLAND HUNT CLUB,OAKLAND HUNT CLUB,BYO Permit,BYO Permit,99,NaN,SOMERTON ROAD,GREENVALE,3047.0,...,144.856975,NaN,NaN,SOMERTON ROAD,GREENVALE,3047,HUME CITY COUNCIL,Metro North,Metro,N
2,31100138,C S I R SKI CLUB,C S I R SKI CLUB,BYO Permit,BYO Permit,99,NaN,SITE 65 1 SCHUSS STREET,FALLS CREEK,3699.0,...,147.278413,NaN,NaN,PO BOX 15,CARNEGIE,3163,ALPINE SHIRE COUNCIL,Hume,Regional,N
3,31100154,SKILIB ALPINE CLUB,SKILIB ALPINE CLUB,BYO Permit,BYO Permit,99,NaN,2 GOAL POSTS RD,MT BULLER,3723.0,...,146.454648,NaN,NaN,PO BOX 140,CANTERBURY,3126,MANSFIELD SHIRE COUNCIL,Hume,Regional,N
4,31100269,THEODOR HERZL SOCIAL CLUB,THEODOR HERZL SOCIAL CLUB,BYO Permit,BYO Permit,99,NaN,222 BALACLAVA RD,CAULFIELD,3162.0,...,145.023196,NaN,NaN,222 BALACLAVA RD,CAULFIELD,3162,GLEN EIRA CITY COUNCIL,Metro South,Metro,N


In [4]:
locations.columns

select_cols = ['Trading As', 'Category', 'Trading Hours','Address', 'Suburb', 'Postcode',

       'Latitude', 'Longitude', 'Postal Postcode', 'Council', 'Region',

       'Metro/Regional', 'Gaming'] #pick the ones that are useful

In [8]:
#categories

pd.unique(locations['Category'])

on_license = ['General Licence', 'Late night (general) Licence', 'Full Club Licence', 'Late night (on-premises) Licence', 'On-Premises Licence']

off_license = ['General Licence', 'Late night (general) Licence', 'Late night (packaged liquor) Licence', 'Late night (general) Licence', 'Packaged Liquor Licence']

In [9]:
#all locations

all_locationsXY = locations[select_cols]

all_locationsXY.to_csv("alcohol_locations.csv")

all_locationsXY.head()

,Trading As,Category,Trading Hours,Address,Suburb,Postcode,Latitude,Longitude,Postal Postcode,Council,Region,Metro/Regional,Gaming
0,UNIVERSITY SKI CLUB,BYO Permit,BYO Permit,U S C LODGE ALLOTMENTS 15 & 16 ALPINE VILL,MT BULLER,3723.0,-37.146305,146.452276,3171,MANSFIELD SHIRE COUNCIL,Hume,Regional,N
1,OAKLAND HUNT CLUB,BYO Permit,BYO Permit,SOMERTON ROAD,GREENVALE,3047.0,-37.631693,144.856975,3047,HUME CITY COUNCIL,Metro North,Metro,N
2,C S I R SKI CLUB,BYO Permit,BYO Permit,SITE 65 1 SCHUSS STREET,FALLS CREEK,3699.0,-36.864882,147.278413,3163,ALPINE SHIRE COUNCIL,Hume,Regional,N
3,SKILIB ALPINE CLUB,BYO Permit,BYO Permit,2 GOAL POSTS RD,MT BULLER,3723.0,-37.146957,146.454648,3126,MANSFIELD SHIRE COUNCIL,Hume,Regional,N
4,THEODOR HERZL SOCIAL CLUB,BYO Permit,BYO Permit,222 BALACLAVA RD,CAULFIELD,3162.0,-37.872303,145.023196,3162,GLEN EIRA CITY COUNCIL,Metro South,Metro,N


In [10]:
#on license

on_licenseXY = locations.loc[locations['Category'].isin(on_license)]

pd.unique(on_licenseXY['Category'])

on_licenseXY.to_csv("on_license_locations.csv")

In [11]:
#off license

off_licenseXY = locations.loc[locations['Category'].isin(off_license)]

pd.unique(off_licenseXY['Category'])

off_licenseXY.to_csv("off_license_locations.csv")